# NLP Lab9 - Contextual question answering

**Author: Bartłomiej Jamiołkowski**

The aim of this exercise is building a neural model able to answer contextual questions in the legal domain.

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from collections import Counter
from random import sample
from tabulate import tabulate
from tqdm import tqdm

import pandas as pd
import numpy as np
import string
import re
import json
import warnings
warnings.filterwarnings('ignore')

## Tasks 1 and 3

Get acquainted with the [Simple legal questions dataset](https://github.com/apohllo/simple-legal-questions-pl) (you need to send your github login to gain access to the dataset).

The legal questions dataset is your **test dataset**.

In [2]:
test_queries_df = pd.read_json('./data/questions.jl', lines = True).set_index('_id').sort_index()
test_queries_df.head()

,text
_id,
1,"Czy żołnierz, który dopuszcza się czynnej napa..."
2,Z ilu osób składa się komisja przetargowa?
3,Do jakiej wysokości za zobowiązania spółki odp...
4,"Kiedy ustala się wartość majątku obrotowego, k..."
5,"Jakiej karze podlega armator, który wykonuje r..."


In [3]:
test_passages_df = pd.read_json('./data/passages.jl', lines = True).set_index('_id').sort_index()
test_passages_df.head()

,title,text
_id,,
1993_599_1,Ustawa z dnia 9 grudnia 1993 r. o zmianie usta...,Art. 1. W ustawie z dnia 8 stycznia 1993 r. o ...
1993_599_2,Ustawa z dnia 9 grudnia 1993 r. o zmianie usta...,Art. 2. W okresie od dnia wejścia w życie usta...
1993_599_3,Ustawa z dnia 9 grudnia 1993 r. o zmianie usta...,Art. 3. Minister Finansów ogłosi w Dzienniku U...
1993_599_4,Ustawa z dnia 9 grudnia 1993 r. o zmianie usta...,Art. 4. Ustawa wchodzi w życie z dniem 1 stycz...
1993_602_1,Ustawa z dnia 10 grudnia 1993 r. o zmianie nie...,Art. 1. W ustawie z dnia 29 maja 1974 r. o zao...


In [4]:
test_answers_df = pd.read_json('./data/answers.jl', lines = True).set_index('question-id').sort_index()
test_answers_df.head()

,score,answer
question-id,,
1,1.0,"Tak, podlega karze aresztu wojskowego albo poz..."
2,1.0,Komisja przetargowa składa się z co najmniej t...
3,1.0,Komandytariusz odpowiada za zobowiązania spółk...
4,1.0,Wartość rzeczowych składników majątku obrotowe...
5,1.0,Podlega karze pieniężnej do wysokości 1 000 00...


In [5]:
test_relevant_df = pd.read_json('./data/relevant.jl', lines = True).set_index('question-id').sort_index()
test_relevant_df.head()

,passage-id,score
question-id,,
1,1997_553_345,1
2,2004_177_21,1
3,1996_465_111,1
4,1994_591_35,1
5,2001_1441_74,1


## Tasks 4 - 5

[PoQuAD](https://huggingface.co/datasets/clarin-pl/poquad) is your **train and validation dataset** (use the splits from the repo).

**Warning** PoQuAD has a python API compatible with the `datasets` library, but it only provides the **extractive answers**, even
   though the abstractive answers are available in the JSON files. So you have to read the JSON files directly.

In [6]:
with open('./data/poquad-train.json', 'r', encoding = 'utf-8') as file:
    train_data = json.load(file)['data']

train_data_df = pd.DataFrame(train_data)
train_data_df.head()

,id,title,summary,url,paragraphs
0,7609,Konfederacja polsko-czechosłowacka,Konfederacja polsko-czechosłowacka – koncepcja...,https://pl.wikipedia.org/wiki/Konfederacja_pol...,[{'context': 'Projekty konfederacji zaczęły si...
1,8420,Pomilio PD/PE,Pomilio PD/PE – rodzina włoskich samolotów roz...,https://pl.wikipedia.org/wiki/Pomilio_PD/PE,[{'context': 'Samoloty Pomilio PD weszły na wy...
2,9163,Edmund Kowal,"Edmund Karol Kowal, w latach 1931–1949 Erwin E...",https://pl.wikipedia.org/wiki/Edmund_Kowal,[{'context': 'Kowal trafił do wojska jesienią ...
3,7492,Sejm Rzeczypospolitej Polskiej,"Sejm Rzeczypospolitej Polskiej (Sejm RP), Sejm...",https://pl.wikipedia.org/wiki/Sejm_Rzeczypospo...,[{'context': 'Zgodnie z Regulaminem Sejmu i za...
4,9713,Amanda Lear,"Amanda Lear – francuska piosenkarka, autorka t...",https://pl.wikipedia.org/wiki/Amanda_Lear,[{'context': 'W 2006 otworzyła wystawę malarsk...


In [7]:
with open('./data/poquad-dev.json', 'r', encoding = 'utf-8') as file:
    validation_data = json.load(file)['data']

validation_data_df = pd.DataFrame(validation_data)
validation_data_df.head()

,id,title,summary,url,paragraphs
0,9773,Miszna,"Miszna (hebr. ‏משנה‎ miszna „nauczać”, „ustnie...",https://pl.wikipedia.org/wiki/Miszna,[{'context': 'Pisma rabiniczne – w tym Miszna ...
1,7478,Emilia Plater,Emilia Broel-Plater herbu Plater (ur. 13 listo...,https://pl.wikipedia.org/wiki/Emilia_Plater,[{'context': 'Sformowany przez nią oddział par...
2,7699,Peter Phillips,"Peter Phillips (Peter Mark Andrew Phillips, ur...",https://pl.wikipedia.org/wiki/Peter_Phillips,[{'context': 'Zaręczyny pary ogłoszono 28 lipc...
3,7815,Karnawał,"Karnawał, zapusty – okres zimowych balów, mask...",https://pl.wikipedia.org/wiki/Karnawał,[{'context': 'Dawniej w zapusty jedzono dużo t...
4,7588,Superpuchar Polski w piłce nożnej,Superpuchar Polski w piłce nożnej (w latach 20...,https://pl.wikipedia.org/wiki/Superpuchar_Pols...,"[{'context': 'Od 2014 roku w Superpucharze, z ..."


## Task 8

If you have problems training the model, you can use [apohllo/plt5-base-poquad](https://huggingface.co/apohllo/plt5-base-poquad) which was trained on PoQuAD.

In [8]:
tokenizer = AutoTokenizer.from_pretrained('apohllo/plt5-base-poquad')
model = AutoModelForSeq2SeqLM.from_pretrained('apohllo/plt5-base-poquad')

## Tasks 9 - 10

Report the obtained performance of the model (in the form of a table). The report should include *exact match* and *F1 score* 
   for the tokens appearing both in the reference and the predicted answer.

Report the best results obtained on the validation dataset and the corresponding results on your test dataset. The results on the 
   test set have to be obtained for the model that yield the best result on the validation dataset.

First, I need to normalize textual data, because: records are from distinct datasets and some columns are not necessary. I hope these process will allow me to save computational resources.

In [9]:
def normalize_data(data: pd.DataFrame) -> pd.DataFrame:
    normalized_data_df = pd.DataFrame(columns = ['queries', 'context', 'answers'])

    for id, row in data.iterrows():
        for paragraph in row['paragraphs']:
            for qa in paragraph['qas']:
                new_row = {
                    'queries': qa['question'],
                    'context': paragraph['context'],
                    'answers': (qa['answers'][0]['generative_answer'] if 'answers' in qa.keys() else qa['plausible_answers'][0]['generative_answer'])
                }
            
                normalized_data_df = pd.concat([normalized_data_df, pd.DataFrame([new_row])], ignore_index = True)

    return normalized_data_df

In [10]:
train_data_df = normalize_data(train_data_df)
train_data_df.head()

,queries,context,answers
0,Co było powodem powrócenia konceptu porozumien...,Projekty konfederacji zaczęły się załamywać 5 ...,wymiana listów Ripka – Stroński
1,Pomiędzy jakimi stronami odbyło się zgromadzen...,Projekty konfederacji zaczęły się załamywać 5 ...,E. Beneš i J. Masaryk a Wł. Sikorski i E. Racz...
2,O co ubiegali się polscy przedstawiciele podcz...,Projekty konfederacji zaczęły się załamywać 5 ...,podpisanie układu konfederacyjnego
3,Który z dyplomatów sprzeciwił się konceptowi k...,Projekty konfederacji zaczęły się załamywać 5 ...,E. Beneš
4,Kiedy oficjalnie doszło do zawarcia porozumienia?,Projekty konfederacji zaczęły się załamywać 5 ...,20 listopada 1942


In [11]:
val_data_df = normalize_data(validation_data_df)
val_data_df.head()

,queries,context,answers
0,Czym są pisma rabiniczne?,Pisma rabiniczne – w tym Miszna – stanowią kom...,kompilacją poglądów różnych rabinów na określo...
1,Z ilu komponentów składała się Tora przekazana...,Pisma rabiniczne – w tym Miszna – stanowią kom...,dwóch
2,W jakich formach występowała Tora przekazana M...,Pisma rabiniczne – w tym Miszna – stanowią kom...,"pisanej, ustnej"
3,W jakiej formie przekazana została Miszna?,Pisma rabiniczne – w tym Miszna – stanowią kom...,ustnej
4,Kto napisał Torę?,Pisma rabiniczne – w tym Miszna – stanowią kom...,Bóg


Unfortunately, test data is provided in different order. Therefore, I cannot use previous normalize_data function.

In [12]:
test_data_df = pd.DataFrame(columns = ['queries', 'context', 'answers'])

for query_id, row in test_relevant_df.iterrows():
    try:
        new_row = {
            'queries': test_queries_df.loc[query_id]['text'],
            'context': test_passages_df.loc[row['passage-id']]['text'],
            'answers': test_answers_df.loc[query_id]['answer']
        }
    
        test_data_df = pd.concat([test_data_df, pd.DataFrame([new_row])], ignore_index = True)
    except:
        pass

Below, are created functions responsible for conducting model evaluation.

In [13]:
def generate_answer(query: str, context: str) -> str:
    tokens = tokenizer.encode(f'Query: {query}. Context: {context}', return_tensors = 'pt').to('cpu')
    answer = model.generate(tokens, max_length = 1000, do_sample = True, top_p = 0.9, top_k = 50)
    answer = tokenizer.decode(answer[0], skip_special_tokens = True)
    return answer

def normalize_answer(answer: str) -> str:
    return ' '.join(re.sub(f'[{string.punctuation}]', '', answer.lower()).split())

def exact_match(pred_answer: str, true_answer: str) -> bool:
    return normalize_answer(pred_answer) == normalize_answer(true_answer)

def f1_score(pred_answer: str, true_answer: str) -> float:
    pred_answer_tokens = normalize_answer(pred_answer).split()
    truth_answer_tokens = normalize_answer(true_answer).split()

    same_tokens = Counter(pred_answer_tokens) & Counter(truth_answer_tokens)
    same_tokens_sum = sum(same_tokens.values())
    
    precision = same_tokens_sum / len(pred_answer_tokens) if len(pred_answer_tokens) > 0 else 0
    recall = same_tokens_sum / len(truth_answer_tokens) if len(truth_answer_tokens) > 0 else 0
    
    if precision == 0 and recall == 0:
        return 0.0
    
    return (2 * precision * recall) / (precision + recall)

def evaluate_model(data: pd.DataFrame) -> tuple[float, float]:
    exact_matches, f1_scores = [], []
    
    for _, row in tqdm(data.iterrows(), desc = 'Evaluating'):
        pred_answer = generate_answer(row['queries'], row['context'])
        exact_matches.append(int(exact_match(pred_answer, row['answers'])))
        f1_scores.append(f1_score(pred_answer, row['answers']))
    
    return np.mean(exact_matches), np.mean(f1_scores)

In [14]:
test_exact_match, test_f1_score = evaluate_model(test_data_df)

Evaluating: 680it [04:32,  2.50it/s]


In [15]:
print(f"Model performance for test dataset\n{tabulate([[test_exact_match, test_f1_score]], headers = ['Exact match', 'F1 score'], tablefmt = 'grid')}")

Model performance for test dataset
+---------------+------------+
|   Exact match |   F1 score |
+===============+============+
|      0.252941 |     0.4415 |
+---------------+------------+


In [16]:
val_exact_match, val_f1_score = evaluate_model(val_data_df)

Evaluating: 7060it [44:06,  2.67it/s]


In [17]:
print(f"Model performance for validation dataset\n{tabulate([[val_exact_match, val_f1_score]], headers = ['Exact match', 'F1 score'], tablefmt='grid')}")

Model performance for validation dataset
+---------------+------------+
|   Exact match |   F1 score |
+===============+============+
|      0.506657 |    0.68258 |
+---------------+------------+


## Task 11

Generate, report and analyze the answers for at least 10 questions provided by the best model on you test dataset.

I decided to generate 15 answers in the frame of this task. However, they will disappear if code is run again.

In [20]:
def print_queries_with_answers(data: pd.DataFrame, n: int) -> None:
    for _, row in data.sample(n = n).iterrows():
        pred_answer = generate_answer(row['queries'], row['context'])
        print(f"Query: {row['queries']}")
        print(f"True answer: {row['answers']}")
        print(f"Predicted answer: {pred_answer}\n{'-' * 45}")

In [32]:
print_queries_with_answers(test_data_df, 15)

Query: Kiedy artykuły spożywcze przywiezione z zagranicy mogą być objętą procedurą dopuszczenia do obrotu?
True answer: Pod wpływem przeprowadzenia kontroli jakości
Predicted answer: pod warunkiem przeprowadzenia kontroli jakości handlowej przez organ Inspekcji Jakości Handlowej Artykułów Rolno-Spożywczych i sporządzeniu protokołu z tej kontroli
---------------------------------------------
Query: Gdzie mogą być przekazywana zatrzymane żywe zwierzęta?
True answer: do ogrodów botanicznych, ogrodów zoologicznych lub ośrodków rehabilitacji zwierząt
Predicted answer: do ogrodów botanicznych, ogrodów zoologicznych lub ośrodków rehabilitacji zwierząt
---------------------------------------------
Query: Jakie dane obejmuje rejestr detektywów?
True answer: imię i nazwisko, adres zamieszkania , numer licencji i datę jej wydania, datę zawieszenia lub cofnięcia licencji
Predicted answer: imię i nazwisko, 2) adres zamieszkania, 3) numer licencji i datę jej wydania, 4) datę zawieszenia lub cofnięci

An analysis of these results is written under Question 2.

### Question 1

Does the performance on the validation dataset reflects the performance on your test set?

The performance on the validation dataset is better than the performance on the test set. This is particularly noticeable in the exact match scores, where the difference between the two scores is significant. Weaker performance was observed on the test dataset because it is specific to a single domain. Perhaps dataset size also played important role.

### Question 2

What are the outcomes of the model on your test questions? Are they satisfying? If not, what might be the reason for that?

Outcomes of the model differ a lot. ome of them are short answers, whereas the others are complex answers. It is worth to mention, that in 15 analyzed answers. I found 7 wrong or partially proper answers. There are many details to discuss so I am going to list them:

- Jakie instytucje są organami dochodzenia w sprawach o przestępstwa skarbowe i wykroczenia? - for this query model returned 2 from 4 expected answers,
- Jakiej karze podlega osoba wybierająca jaja ptaków łownych? - for this query model returned wrong answer, because the punishment is not 5 years of prison, but rather so called 'grzywna' or 'nagana',
- W jaki sposób odpowiada sędzia za wykroczenia? - surprisingly model returned better answer than provided answer, because 'dyscyplinarnie' is an exact answers, whereas provided answer is not exactly an answer itself,
- Od czego rozpoczyna się przewód sądowy? - model returned wrong answer,
- Czy udziałowcy mogą ponieść  odpowiedzialność za straty powstałe w banku? - model returned wrong answer,
- Czy w każdej gminie znajduje się wojewódzka biblioteka publiczna? - model returned wrong answer, comparing to provided answer and my general knowledge,
- Co grozi osobie, która prowadzi obrót detaliczny produktami leczniczymi weterynaryjnymi wydawanymi bez przepisu lekarza? - model returned partially proper answer.

Overall, I am satisfied with the outcomes, despite minor mistakes. Perhaps model parameters have impact on obtained results.

### Question 3

Why extractive question answering is not well suited for inflectional languages?

Extractive question answering is not well suited for inflectional languages due to their complex word forms and extensive grammatical variations. In these languages, a single word can take many forms depending on its grammatical role. This variability makes aligning questions with answers in the text
difficullt for extractive systems, because they rely on exact or near-exact keyword matching. As a result, direct keyword matching becomes unreliable.